<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Assignment-->-Draw-a-network-on-and-code-it" data-toc-modified-id="Assignment-->-Draw-a-network-on-and-code-it-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Assignment--&gt; Draw a network on and code it</a></span><ul class="toc-item"><li><span><a href="#building-UNet.-Here-is-the-overall-architecture" data-toc-modified-id="building-UNet.-Here-is-the-overall-architecture-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>building UNet. Here is the overall architecture</a></span></li></ul></li></ul></div>

## Assignment--> Draw a network on and code it

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import tqdm, time

import glob
import random,time
import math

from functools import partial

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import *
from tensorflow.keras.regularizers import *

### building UNet. Here is the overall architecture

![alt](asset/unet.jpg)

A UNet consists of an encoder (downsampler) and decoder (upsampler) with a bottleneck in between. The black arrows correspond to skip connections that concatenate encoder block outputs to each stage of the decoder

encoder utilities will have three functions:

conv2d_block() - to add two convolution layers and ReLU activations

encoder_block() - to add pooling and dropout to the conv2d blocks. in UNet need to save the output of the convolution layers at each block so this function will return two values to take that into account (i.e. output of the conv block and the dropout)

encoder() - to build the entire encoder. This will return the output of the last encoder block as well as the output of the previous conv blocks. These will be concatenated to the decoder blocks

In [5]:
# Encoder Utilities

def conv2d_block(input_tensor, n_filters, kernel_size = 3):

        # first layer
        x = input_tensor
        for i in range(2):
            x = tf.keras.layers.Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
                    kernel_initializer = 'he_normal', padding = 'same')(x)
            x = tf.keras.layers.Activation('relu')(x)

        return x

def encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3):
       
        f = conv2d_block(inputs, n_filters=n_filters)
        p = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(f)
        p = tf.keras.layers.Dropout(0.3)(p)

        return f, p

def encoder(inputs):
        
        f1, p1 = encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3)
        f2, p2 = encoder_block(p1, n_filters=128, pool_size=(2,2), dropout=0.3)
        f3, p3 = encoder_block(p2, n_filters=256, pool_size=(2,2), dropout=0.3)
        f4, p4 = encoder_block(p3, n_filters=512, pool_size=(2,2), dropout=0.3)

        return p4, (f1, f2, f3, f4)

A bottleneck follows the encoder block and is used to extract more features. This does not have a pooling layer so the dimensionality remains the same

In [6]:
def bottleneck(inputs):
        '''
        This function defines the bottleneck convolutions to extract more features before the upsampling layers.
        '''
        bottle_neck = conv2d_block(inputs, n_filters=1024)
        return bottle_neck

In decoder which upsamples the features back to the original image size. At each upsampling level, need to take the output of the corresponding encoder block and concatenate it before feeding to the next decoder block

In [8]:
# Decoder Utilities

def decoder_block(inputs, conv_output, n_filters=64, kernel_size=3, strides=3, dropout=0.3):
            
            u = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size, strides = strides, padding = 'same')(inputs)
            c = tf.keras.layers.concatenate([u, conv_output])
            c = tf.keras.layers.Dropout(dropout)(c)
            c = conv2d_block(c, n_filters, kernel_size=3)

            return c


def decoder(inputs, convs, output_channels):

        f1, f2, f3, f4 = convs

        c6 = decoder_block(inputs, f4, n_filters=512, kernel_size=(3,3), strides=(2,2), dropout=0.3)
        c7 = decoder_block(c6, f3, n_filters=256, kernel_size=(3,3), strides=(2,2), dropout=0.3)
        c8 = decoder_block(c7, f2, n_filters=128, kernel_size=(3,3), strides=(2,2), dropout=0.3)
        c9 = decoder_block(c8, f1, n_filters=64, kernel_size=(3,3), strides=(2,2), dropout=0.3)

        outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='softmax')(c9)

        return outputs

In [14]:
OUTPUT_CHANNELS = 3

def unet():
    inputs = tf.keras.layers.Input(shape=(128, 128,3,))

    # feed the inputs to the encoder
    encoder_output, convs = encoder(inputs)

    # feed the encoder output to the bottleneck
    bottle_neck = bottleneck(encoder_output)

    # feed the bottleneck and encoder block outputs to the decoder
    # specify the number of classes via the `output_channels` argument
    outputs = decoder(bottle_neck, convs, output_channels=OUTPUT_CHANNELS)

    # create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# instantiate the model
model = unet()

# see the resulting model architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activation)        (None, 128, 128, 64  0           ['conv2d[0][0]']                 
                                )                                                             

 conv2d_transpose_1 (Conv2DTran  (None, 32, 32, 256)  1179904    ['activation_11[0][0]']          
 spose)                                                                                           
                                                                                                  
 concatenate_1 (Concatenate)    (None, 32, 32, 512)  0           ['conv2d_transpose_1[0][0]',     
                                                                  'activation_5[0][0]']           
                                                                                                  
 dropout_5 (Dropout)            (None, 32, 32, 512)  0           ['concatenate_1[0][0]']          
                                                                                                  
 conv2d_12 (Conv2D)             (None, 32, 32, 256)  1179904     ['dropout_5[0][0]']              
                                                                                                  
 activatio